# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-c481ceaa8e-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Presidents of countries") 

Consider the following exploratory information need:

> You investigating presidents of the republic, or similar roles, across international states around the world

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wd:Q248577`  | President of the republic | node      |
| `wd:Q11696`   | President of U.S.A.      | node      |
| `wd:Q332711`  | President of Italy        | node |
| `wd:Q38`      | Italy          | node |
| `wd:Q30`      | U.S.A.        | node |



Also consider

```
?p wdt:P39/wdt:P279* wd:Q248577  .  
```

is the BGP to retrieve all **presidents of the world countries through history**

## Workload Goals

1. Identify the BGP for obtaining important attributes for people, e.g., date of birth/death, gender, profession    

2. Identify the BGP to retrieve countries with  had at least once had a president     

3. When was the first president of each country born? How many presidents each country had?   **Primo punto forse no... Secondo OK**

4. Is there a country that had at some point a woman as a president? 

5. Analyze the number of presidents per country through time 
 
   5.1 What are the top-5 countries for number of presidents? Which countries had the least?
   
   5.2 For how many presidents we know a date of death in each country?
   
   5.3 Which are the professions of different presidents? How many presidents had a specific profession?


In [1]:
# start your workflow here

In [69]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P39/wdt:P279* wd:Q248577  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1649')]


1

**1 - BGP attributes of a person**

Looking for a president

In [7]:
queryString = """
SELECT ?p ?pName
WHERE { 

?p wdt:P39/wdt:P279* wd:Q248577  . 

?p <http://schema.org/name> ?pName .
} 

LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q14345'), ('pName', 'Roh Moo-hyun')]
[('p', 'http://www.wikidata.org/entity/Q472323'), ('pName', 'Ranasinghe Premadasa')]
[('p', 'http://www.wikidata.org/entity/Q193062'), ('pName', 'Raúl Alfonsín')]
[('p', 'http://www.wikidata.org/entity/Q382830'), ('pName', 'Delfim Moreira')]
[('p', 'http://www.wikidata.org/entity/Q81215'), ('pName', 'Petar Stoyanov')]
[('p', 'http://www.wikidata.org/entity/Q3514836'), ('pName', 'Tammam Salam')]
[('p', 'http://www.wikidata.org/entity/Q15029'), ('pName', 'Hu Jintao')]
[('p', 'http://www.wikidata.org/entity/Q206479'), ('pName', 'Émile Lahoud')]
[('p', 'http://www.wikidata.org/entity/Q57478'), ('pName', 'Nicolae Nasoc')]
[('p', 'http://www.wikidata.org/entity/Q57634'), ('pName', 'Hashim Thaçi')]
[('p', 'http://www.wikidata.org/entity/Q123923'), ('pName', 'Radovan Karadžić')]
[('p', 'http://www.wikidata.org/entity/Q1148669'), ('pName', 'Matamela Cyril Ramaphosa')]
[('p', 'http://www.wikidata.org/entity/Q2097

15

Exploring Petar Stoyanov Q81215 to get his properties

In [12]:
queryString = """
SELECT ?p ?pName ?obj ?objName
WHERE { 

wd:Q81215 ?p ?obj  . 

OPTIONAL {?p <http://schema.org/name> ?pName}
OPTIONAL {?obj <http://schema.org/name> ?objName}


} 

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q82955'), ('objName', 'politician')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('pName', 'member of political party'), ('obj', 'http://www.wikidata.org/entity/Q841253'), ('objName', 'Union of Democratic Forces')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q82955'), ('objName', 'politician')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q40348'), ('objName', 'lawyer')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q67311526'), ('objName', 'Obalky knih.cz')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName', 'languages spoken, written or signed'), ('obj', 'http://www.wikidata.org/entity/Q7918'), ('objName', 'Bulgarian

53

I've found many important properties and information about the person, like gender P21, born date P569, profession P106 and position held P39

Looking for the death date with a REGEX

In [24]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

?person wdt:P39/wdt:P279* wd:Q248577  .
?person ?p ?obj  . 

?p <http://schema.org/name> ?pName.

FILTER (REGEX(?pName, ".*death.*"))
} 

LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('pName', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('pName', 'place of death')]
[('p', 'http://www.wikidata.org/prop/direct/P509'), ('pName', 'cause of death')]
[('p', 'http://www.wikidata.org/prop/direct/P570'), ('pName', 'date of death')]


4

In [44]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 
        
?person wdt:P31 wd:Q5 ;
         ?p ?obj . 
         

?p <http://schema.org/name> ?pName .


OPTIONAL {?obj <http://schema.org/name> ?objName .}


            # birth - death -profession/occupation
            FILTER ((?p = wdt:P569 OR ?p = wdt:P570) OR (?p = wdt:P101 OR ?p = wdt:P106 ) OR (?p = wdt:P21))
} 
LIMIT 15
"""
#
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q11497309'), ('objName', 'economic support')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q121713'), ('objName', 'acupuncture')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q1673980'), ('objName', 'Smithsonian Tropical Research Institute')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q19577'), ('objName', 'actinide')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q229438'), ('objName', '3D television')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q233708'), ('objName', 'Viking program')]
[

15

I've found date of death P570

**Answer 1**

```
        
?person wdt:P31 wd:Q5 ;
         ?p ?obj . 
         
?p <http://schema.org/name> ?pName .


OPTIONAL {?obj <http://schema.org/name> ?objName .}


            # birth - death -profession/occupation
            FILTER ((?p = wdt:P569 OR ?p = wdt:P570) OR (?p = wdt:P101 OR ?p = wdt:P106 ) OR (?p = wdt:P21))

```

Using it on a person (president) found before

In [47]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 

    wd:Q20978710 ?p ?obj . 
         

?p <http://schema.org/name> ?pName .


OPTIONAL {?obj <http://schema.org/name> ?objName .}


            # birth - death -profession/occupation
            FILTER ((?p = wdt:P569 OR ?p = wdt:P570) OR (?p = wdt:P101 OR ?p = wdt:P106 ) OR (?p = wdt:P21))
} 
LIMIT 15
"""
#
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q47064'), ('objName', 'military personnel')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q82955'), ('objName', 'politician')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender'), ('obj', 'http://www.wikidata.org/entity/Q6581097'), ('objName', 'male')]
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('pName', 'date of birth'), ('obj', '1960-01-01T00:00:00Z')]


4

---------------------------------------

**2 - Identify the BGP to retrieve countries with  had at least once had a president**

Now I can explot the properties P27 country of citizenship or P39 position held. 

Esploro Q219 Bulgary e Q14946265 president of bulgaria

In [4]:
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE { 

wd:Q14946265 ?p ?obj .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}

} 
ORDER BY ASC (?pName)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('obj', 'Presidents of Bulgaria')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('obj', '/m/012z31w1')]
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction'), ('obj', 'http://www.wikidata.org/entity/Q219'), ('objName', 'Bulgaria')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('obj', 'http://www.wikidata.org/entity/Q127403'), ('objName', 'list of presidents of Bulgaria')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pName', 'inception'), ('obj', '1990-04-03T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q4164871'), ('objName', 'position')]
[('p', 'http://www.wikidata.org/prop/direct/P1308'), ('pName', 'officeholder'), ('obj', 'http://www.wikidata.org/entity/Q26257557'), ('objName', 'Rumen Radev')]
[('p', 'ht

13

In [ ]:
Inverting the path

In [5]:
queryString = """
SELECT DISTINCT ?obj ?objName ?p ?pName 
WHERE { 

?obj ?p wd:Q14946265 .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?obj <http://schema.org/name> ?objName .}

} 
ORDER BY ASC (?pName)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q127403'), ('objName', 'list of presidents of Bulgaria'), ('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of')]
[('obj', 'http://www.wikidata.org/entity/Q27963665'), ('objName', 'April 1990 Bulgarian presidential election'), ('p', 'http://www.wikidata.org/prop/direct/P541'), ('pName', 'office contested')]
[('obj', 'http://www.wikidata.org/entity/Q1993031'), ('objName', '2006 Bulgarian presidential election'), ('p', 'http://www.wikidata.org/prop/direct/P541'), ('pName', 'office contested')]
[('obj', 'http://www.wikidata.org/entity/Q30634376'), ('objName', '2021 Bulgarian presidential election'), ('p', 'http://www.wikidata.org/prop/direct/P541'), ('pName', 'office contested')]
[('obj', 'http://www.wikidata.org/entity/Q2159895'), ('objName', '2001 Bulgarian presidential election'), ('p', 'http://www.wikidata.org/prop/direct/P541'), ('pName', 'office contested')]
[('obj', 'http://www.wikidata.org/entity/Q23022170'), (

15

Now with P27 country of citizenship.

In [26]:
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?country ?p ?obj .


OPTIONAL {?p <http://schema.org/name> ?pName.}
OPTIONAL {?country <http://schema.org/name> ?countryName.}
} 
ORDER BY (?countryName)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q222'), ('countryName', 'Albania')]
[('country', 'http://www.wikidata.org/entity/Q777147'), ('countryName', 'Albanian Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q1923781'), ('countryName', 'Albanian Republic')]
[('country', 'http://www.wikidata.org/entity/Q262'), ('countryName', 'Algeria')]
[('country', 'http://www.wikidata.org/entity/Q916'), ('countryName', 'Angola')]
[('country', 'http://www.wikidata.org/entity/Q146600'), ('countryName', 'Arab Kingdom of Syria')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q399'), ('countryName', 'Armenia')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q28513'), ('countryName', 'Austria-Hungary')]
[('country', 'http://www.wikidata.org/entity/Q227'), ('countryName', 'Azerbaijan')]
[('country', 'http://www.wikidata.org/entity/

20

Counting to have an idea

In [10]:
queryString = """
SELECT COUNT(DISTINCT (?country))
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?country ?p ?obj .

?country <http://schema.org/name> ?countryName.
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '206')]


1

**Answer 2**

Finally I can answer to the second point

```

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?country ?p ?obj .

?country <http://schema.org/name> ?countryName.
} 
```

I can understand that at least 200 countries had a president

In [27]:
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?country ?p ?obj .

?country <http://schema.org/name> ?countryName.
} 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q212056'), ('countryName', "Mongolian People's Republic")]
[('country', 'http://www.wikidata.org/entity/Q899706'), ('countryName', 'Peru–Bolivian Confederation')]
[('country', 'http://www.wikidata.org/entity/Q1061488'), ('countryName', 'United States of Colombia')]
[('country', 'http://www.wikidata.org/entity/Q268970'), ('countryName', 'Republic of German-Austria')]
[('country', 'http://www.wikidata.org/entity/Q241748'), ('countryName', 'Kingdom of Serbia')]
[('country', 'http://www.wikidata.org/entity/Q907112'), ('countryName', 'Transnistria')]
[('country', 'http://www.wikidata.org/entity/Q12183911'), ('countryName', 'Syrian Federation')]
[('country', 'http://www.wikidata.org/entity/Q146600'), ('countryName', 'Arab Kingdom of Syria')]
[('country', 'http://www.wikidata.org/entity/Q146880'), ('countryName', 'State of Syria')]
[('country', 'http://www.wikidata.org/entity/Q146885'), ('countryName', 'Second Syrian Republic')]


10

-----------------
**3- When was the first president of each country born? How many presidents each country had?**


I try to understand how the string are processed

In [52]:
queryString = """
SELECT ?birth1 ?birth2 WHERE { 

wd:Q20978710 wdt:P569 ?birth1  .
wd:Q81215 wdt:P569 ?birth2 .


} 

"""

print("Results")
run_query(queryString)

Results
[('birth1', '1960-01-01T00:00:00Z'), ('birth2', '1952-05-25T00:00:00Z')]


1

In [131]:
queryString = """
ASK WHERE { 

wd:Q20978710 wdt:P569 ?birth1  .
wd:Q81215 wdt:P569 ?birth2 .

FILTER (?birth1 < ?birth2)
} 

"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': False}

The comparson works correctly

Now with the born date?

In [5]:
queryString = """
SELECT DISTINCT ?countryName (MIN(?b) as ?birth_date) 
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?president wdt:P569 ?birth .

BIND(STR(?birth) as ?b)

?country <http://schema.org/name> ?countryName.

}
GROUP BY  ?countryName ?birth
ORDER BY ASC (?countryName)
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Albania'), ('birth_date', '1944-10-15T00:00:00Z')]
[('countryName', 'Albania'), ('birth_date', '1966-09-29T00:00:00Z')]
[('countryName', 'Albania'), ('birth_date', '1913-10-19T00:00:00Z')]
[('countryName', 'Albania'), ('birth_date', '1969-03-24T00:00:00Z')]
[('countryName', 'Albania'), ('birth_date', '1957-04-24T00:00:00Z')]
[('countryName', 'Albania'), ('birth_date', '1929-12-01T00:00:00Z')]
[('countryName', 'Albania'), ('birth_date', '1944-08-17T00:00:00Z')]
[('countryName', 'Albania'), ('birth_date', '1887-02-05T00:00:00Z')]
[('countryName', 'Albania'), ('birth_date', '1844-10-16T00:00:00Z')]
[('countryName', 'Albania'), ('birth_date', '1925-10-18T00:00:00Z')]
[('countryName', 'Albanian Kingdom'), ('birth_date', '1895-10-08T00:00:00Z')]
[('countryName', 'Albanian Republic'), ('birth_date', '1895-10-08T00:00:00Z')]
[('countryName', 'Algeria'), ('birth_date', '1885-01-01T00:00:00Z')]
[('countryName', 'Angola'), ('birth_date', '1942-08-28T00:00:00Z')]
[('count

1912

It return all the dates of a country... It is wrong.

Trying to remove the country name

In [125]:
queryString = """
SELECT  (MIN(?birth) as ?birth_date) ?country
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?president wdt:P569 ?birth .

}
GROUP BY  ?birth_date ?country
ORDER BY ASC (?country)
"""

print("Results")
run_query(queryString)

Results
[('birth_date', '1924-05-16T00:00:00Z'), ('country', 'http://www.wikidata.org/entity/Q1005')]
[('birth_date', '1922-01-09T00:00:00Z'), ('country', 'http://www.wikidata.org/entity/Q1006')]
[('birth_date', '1906-04-13T00:00:00Z'), ('country', 'http://www.wikidata.org/entity/Q1019')]
[('birth_date', '1920-10-20T00:00:00Z'), ('country', 'http://www.wikidata.org/entity/Q1027')]
[('birth_date', '1933-09-29T00:00:00Z'), ('country', 'http://www.wikidata.org/entity/Q1029')]
[('birth_date', '1929-05-12T00:00:00Z'), ('country', 'http://www.wikidata.org/entity/Q1030')]
[('birth_date', '1904-11-16T00:00:00Z'), ('country', 'http://www.wikidata.org/entity/Q1033')]
[('birth_date', '1912-04-10T00:00:00Z'), ('country', 'http://www.wikidata.org/entity/Q1036')]
[('birth_date', '1935-01-01T00:00:00Z'), ('country', 'http://www.wikidata.org/entity/Q1039')]
[('birth_date', '1935-11-16T00:00:00Z'), ('country', 'http://www.wikidata.org/entity/Q1042')]
[('birth_date', '1908-12-09T00:00:00Z'), ('country',

205

It works! 
Now I use the previous query as subquery. In the outer query I search for the president name

In [135]:
queryString = """

SELECT DISTINCT ?countryName ?presidentName ?president ?birthd WHERE{


?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?president wdt:P569 ?birthd .

?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?presidentName.

FILTER (?birthd = ?birth_date)

            {
SELECT  (MIN(?birth) as ?birth_date) ?country
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?president wdt:P569 ?birth .

}
GROUP BY  ?birth_date ?country
ORDER BY ASC (?country)


            }

        
}
ORDER BY ASC (?countryName)

"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Albania'), ('presidentName', 'Ismail Qemali'), ('president', 'http://www.wikidata.org/entity/Q46932'), ('birthd', '1844-10-16T00:00:00Z')]
[('countryName', 'Albanian Kingdom'), ('presidentName', 'Zog I'), ('president', 'http://www.wikidata.org/entity/Q60699'), ('birthd', '1895-10-08T00:00:00Z')]
[('countryName', 'Albanian Republic'), ('presidentName', 'Zog I'), ('president', 'http://www.wikidata.org/entity/Q60699'), ('birthd', '1895-10-08T00:00:00Z')]
[('countryName', 'Angola'), ('presidentName', 'Lúcio Lara'), ('president', 'http://www.wikidata.org/entity/Q1479462'), ('birthd', '1929-04-09T00:00:00Z')]
[('countryName', 'Arab Kingdom of Syria'), ('presidentName', 'Habib Pacha Es-Saad'), ('president', 'http://www.wikidata.org/entity/Q880705'), ('birthd', '1867-01-01T00:00:00Z')]
[('countryName', 'Argentina'), ('presidentName', 'José de San Martín'), ('president', 'http://www.wikidata.org/entity/Q134160'), ('birthd', '1778-02-25T00:00:00Z')]
[('countryName', 'Ar

204

Unfortunately I see that Einaudi is in the list as first president of Italy, but Enrico de Nicola was the first one. I have to modify the query and search better. 

Property of obj connected to 1996 President of Bulgaria

In [132]:
queryString = """
SELECT ?p ?pName
WHERE { 

?s ?p wd:Q2375412 .

?p <http://schema.org/name> ?pName.

}

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P3602'), ('pName', 'candidacy in election')]


3

In [133]:
queryString = """
SELECT ?p ?pName
WHERE { 

wd:Q2375412 ?p ?o .

?p <http://schema.org/name> ?pName.

}

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P541'), ('pName', 'office contested')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('pName', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P726'), ('pName', 'candidate')]
[('p', 'http://www.wikidata.org/prop/direct/P991'), ('pName', 'successful candidate')]


10

Successful candidate P991! mi aspetto di trovare il nome di petar. 

In [134]:
queryString = """
SELECT ?o ?oName
WHERE { 

wd:Q2375412 wdt:P991 ?o .

?o <http://schema.org/name> ?oName.

}

"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q81215'), ('oName', 'Petar Stoyanov')]


1

Moving to Italy. Q1242 Einaudi

In [140]:
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE { 

wd:Q1242 ?p ?o .

?p <http://schema.org/name> ?pName.
OPTIONAL{?o <http://schema.org/name> ?oName.}

}
ORDER BY ASC (?p)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('pName', 'Portuguese National Library ID'), ('o', '39615')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pName', 'Nationale Thesaurus voor Auteurs ID'), ('o', '070464480')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('pName', 'Vatican Library ID (former scheme)'), ('o', 'ADV10207499')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('pName', 'member of political party'), ('o', 'http://www.wikidata.org/entity/Q508733'), ('oName', 'Italian Liberal Party')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q121594'), ('oName', 'professor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q188094'), ('oName', 'economist')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q1930187'), ('oName', 'journalist')]
[(

145

proprietà dei presidenti -generale

In [144]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

?person wdt:P39/wdt:P279* wd:Q248577  .
?person ?p ?obj  . 

?p <http://schema.org/name> ?pName.

} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8992'), ('pName', '1914-1918-Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4114'), ('pName', 'ADK member ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5463'), ('pName', 'AE member ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2342'), ('pName', 'AGORHA person/institution ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4432'), ('pName', 'AKL Online artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9267'), ('pName', 'ANPI person numeric ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9226'), ('pName', 'ARAE ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9791'), ('pName', 'ASE person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8219'), ('pName', 'ASUT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9020'), ('pName', 'AV Production person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5527'), ('pName', 'Academia Brasileira de Letras ID')]
[('p', 'http://www.wikidata.org/prop/direct/P670

904

Edition of Einaudi, "1948 Italian presidential election"

In [157]:
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE { 

wd:Q3722096 ?p ?o . 

OPTIONAL {?p <http://schema.org/name> ?pName.}
OPTIONAL {?o <http://schema.org/name> ?oName.}

} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('o', 'http://g.co/kg/m/012vzt3x')]
[('p', 'http://schema.org/name'), ('o', '1948 Italian presidential election')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('o', '/m/012vzt3x')]
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P726'), ('pName', 'candidate'), ('o', 'http://www.wikidata.org/entity/Q1242'), ('oName', 'Luigi Einaudi')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('pName', 'end time'), ('o', '1948-05-11T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', '

14

In [ ]:
START TIME potrebbe aiutare a trovare il primo presidente

In [3]:
queryString = """
SELECT DISTINCT  ?person ?perName
WHERE { 

?person wdt:P3602 ?election .
?election wdt:P17 wd:Q38 ;
        wdt:P991 ?person ;
        wdt:P580 ?startime .

?person <http://schema.org/name> ?perName.

} 
ORDER BY ASC (?startime)
"""
 
print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1242'), ('perName', 'Luigi Einaudi')]
[('person', 'http://www.wikidata.org/entity/Q1241'), ('perName', 'Giovanni Gronchi')]
[('person', 'http://www.wikidata.org/entity/Q1240'), ('perName', 'Antonio Segni')]
[('person', 'http://www.wikidata.org/entity/Q1238'), ('perName', 'Giuseppe Saragat')]
[('person', 'http://www.wikidata.org/entity/Q1235'), ('perName', 'Giovanni Leone')]
[('person', 'http://www.wikidata.org/entity/Q1233'), ('perName', 'Sandro Pertini')]
[('person', 'http://www.wikidata.org/entity/Q1228'), ('perName', 'Oscar Luigi Scalfaro')]
[('person', 'http://www.wikidata.org/entity/Q1220'), ('perName', 'Giorgio Napolitano')]
[('person', 'http://www.wikidata.org/entity/Q3956186'), ('perName', 'Sergio Mattarella')]
[('person', 'http://www.wikidata.org/entity/Q13581225'), ('perName', 'Luigi Di Maio')]


10

It is wrong! Di Maio is not a republic president... Using  as entity Q248577

In [4]:
queryString = """
SELECT DISTINCT  ?person ?perName
WHERE { 

?person wdt:P39/wdt:P279* wd:Q248577  .
?person wdt:P3602 ?election .
?election wdt:P17 wd:Q38 ;
        wdt:P991 ?person ;
        wdt:P580 ?startime .

?person <http://schema.org/name> ?perName.

} 
ORDER BY ASC (?startime)
"""
print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1242'), ('perName', 'Luigi Einaudi')]
[('person', 'http://www.wikidata.org/entity/Q1241'), ('perName', 'Giovanni Gronchi')]
[('person', 'http://www.wikidata.org/entity/Q1240'), ('perName', 'Antonio Segni')]
[('person', 'http://www.wikidata.org/entity/Q1238'), ('perName', 'Giuseppe Saragat')]
[('person', 'http://www.wikidata.org/entity/Q1235'), ('perName', 'Giovanni Leone')]
[('person', 'http://www.wikidata.org/entity/Q1233'), ('perName', 'Sandro Pertini')]
[('person', 'http://www.wikidata.org/entity/Q1228'), ('perName', 'Oscar Luigi Scalfaro')]
[('person', 'http://www.wikidata.org/entity/Q1220'), ('perName', 'Giorgio Napolitano')]
[('person', 'http://www.wikidata.org/entity/Q3956186'), ('perName', 'Sergio Mattarella')]


9

In [159]:
queryString = """
SELECT DISTINCT  ?person ?perName
WHERE { 
                            #president of italy
?person wdt:P39/wdt:P279* wd:Q332711  .  

?person <http://schema.org/name> ?perName.

} 

"""
 
print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1220'), ('perName', 'Giorgio Napolitano')]
[('person', 'http://www.wikidata.org/entity/Q1224'), ('perName', 'Carlo Azeglio Ciampi')]
[('person', 'http://www.wikidata.org/entity/Q1228'), ('perName', 'Oscar Luigi Scalfaro')]
[('person', 'http://www.wikidata.org/entity/Q1241'), ('perName', 'Giovanni Gronchi')]
[('person', 'http://www.wikidata.org/entity/Q1230'), ('perName', 'Francesco Cossiga')]
[('person', 'http://www.wikidata.org/entity/Q1238'), ('perName', 'Giuseppe Saragat')]
[('person', 'http://www.wikidata.org/entity/Q1245'), ('perName', 'Enrico de Nicola')]
[('person', 'http://www.wikidata.org/entity/Q1233'), ('perName', 'Sandro Pertini')]
[('person', 'http://www.wikidata.org/entity/Q3956186'), ('perName', 'Sergio Mattarella')]
[('person', 'http://www.wikidata.org/entity/Q1242'), ('perName', 'Luigi Einaudi')]
[('person', 'http://www.wikidata.org/entity/Q1235'), ('perName', 'Giovanni Leone')]
[('person', 'http://www.wikidata.org/e

12

Proprietà dei presidenti italiani

In [5]:
queryString = """
SELECT DISTINCT  ?p ?pName
WHERE { 

?person wdt:P39/wdt:P279* wd:Q332711  .
wd:Q332711 ?p ?o . 

?p <http://schema.org/name> ?pName.

} 
"""
 
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('pName', 'official website')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('pName', 'flag')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pName', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2097'), ('pName', 'term length of office')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2389'), ('pName', 'organization directed by the office or position')]
[('p', 'http://www.wikidata.org/prop/direct/P41'), ('pName', 'flag image')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'ht

19

Esploro Cossiga

In [6]:
queryString = """
SELECT DISTINCT  ?p ?pName ?o
WHERE {  

wd:Q1230 ?p ?o . 

?p <http://schema.org/name> ?pName.
OPTIONAL {?o <http://schema.org/name> ?oName.}

} 
ORDER BY ASC (?pName)
"""
 
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5731'), ('pName', 'Angelicum ID'), ('o', '84226')]
[('p', 'http://www.wikidata.org/prop/direct/P950'), ('pName', 'Biblioteca Nacional de España ID'), ('o', 'XX896008')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('pName', 'Bibliothèque nationale de France ID'), ('o', '123197724')]
[('p', 'http://www.wikidata.org/prop/direct/P1986'), ('pName', 'Biographical Dictionary of Italian People ID'), ('o', 'francesco-cossiga')]
[('p', 'http://www.wikidata.org/prop/direct/P5019'), ('pName', 'Brockhaus Enzyklopädie online ID'), ('o', 'cossiga-francesco')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('pName', 'CONOR.SI ID'), ('o', '132143715')]
[('p', 'http://www.wikidata.org/prop/direct/P3935'), ('pName', 'Chamber of Deputies of Italy storia ID'), ('o', 'francesco-cossiga-19280726')]
[('p', 'http://www.wikidata.org/prop/direct/P7865'), ('pName', 'CoBiS author ID'), ('o', '95a5yja38danygu695b5yc1t6gu32c8')]
[('p', 'http://www.wi

128

In [ ]:
As last chance I explore P39, position held

In [181]:
queryString = """
SELECT DISTINCT  ?p ?pName ?oName
WHERE { 

wd:P39 ?p ?o . 

?p <http://schema.org/name> ?pName.
OPTIONAL {?o <http://schema.org/name> ?oName.}

} 
ORDER BY ASC (?pName)
"""
 
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('oName', 'job')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('oName', 'position')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('oName', 'mandate')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('oName', 'political coalition')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('oName', 'by position held')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('oName', 'Frederick II, Holy Roman Emperor')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('oName', 'George Washington')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('oName', 'Barac

29

In [180]:
queryString = """
SELECT DISTINCT  ?s ?sName ?p ?pName 
WHERE { 

?s ?p wd:P39 . 

?p <http://schema.org/name> ?pName.
OPTIONAL {?s <http://schema.org/name> ?sName.}

} 
ORDER BY ASC (?pName)
"""
 
print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q192581'), ('sName', 'job'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q4164871'), ('sName', 'position'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q845436'), ('sName', 'mandate'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q6138528'), ('sName', 'political coalition'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q65970006'), ('sName', 'position held by'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q66794259'), ('sName', 'by position held'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wiki

31

In [8]:
queryString = """
SELECT DISTINCT  ?p ?pName 
WHERE { 

 ?s ?p wd:P3602  . 

?p <http://schema.org/name> ?pName.
OPTIONAL {?o <http://schema.org/name> ?oName.}

} 
ORDER BY ASC (?pName)
"""
 
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1696'), ('pName', 'inverse property')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also')]


4

In [26]:
queryString = """
SELECT DISTINCT  ?person ?perName ?election
WHERE { 
                            #president of italy
?person wdt:P39/wdt:P279* wd:Q332711  ;
        wdt:P3602 ?election . 
?election wdt:P991 ?person .
?election ?p ?o . 

?person <http://schema.org/name> ?perName.

} 

"""
 
print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1220'), ('perName', 'Giorgio Napolitano'), ('election', 'http://www.wikidata.org/entity/Q3586606')]
[('person', 'http://www.wikidata.org/entity/Q1224'), ('perName', 'Carlo Azeglio Ciampi'), ('election', 'http://www.wikidata.org/entity/Q3586608')]
[('person', 'http://www.wikidata.org/entity/Q1220'), ('perName', 'Giorgio Napolitano'), ('election', 'http://www.wikidata.org/entity/Q588041')]
[('person', 'http://www.wikidata.org/entity/Q1228'), ('perName', 'Oscar Luigi Scalfaro'), ('election', 'http://www.wikidata.org/entity/Q3586607')]
[('person', 'http://www.wikidata.org/entity/Q1245'), ('perName', 'Enrico de Nicola'), ('election', 'http://www.wikidata.org/entity/Q3722093')]
[('person', 'http://www.wikidata.org/entity/Q1245'), ('perName', 'Enrico de Nicola'), ('election', 'http://www.wikidata.org/entity/Q3722095')]
[('person', 'http://www.wikidata.org/entity/Q1242'), ('perName', 'Luigi Einaudi'), ('election', 'http://www.wikidata.org/en

14

proprietà dell elezioni pdr italiane

In [27]:
queryString = """
SELECT DISTINCT  ?p ?pName
WHERE { 
                            #president of italy
?person wdt:P39/wdt:P279* wd:Q332711  ;
        wdt:P3602 ?election . 
?election wdt:P991 ?person .
?election ?p ?o . 

?p <http://schema.org/name> ?pName.

} 

"""
 
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P541'), ('pName', 'office contested')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('pName', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('pName', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('pName', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/d

14

In [ ]:
Cossiga election

In [30]:
queryString = """
SELECT DISTINCT  ?p ?pName ?obj
WHERE { 

wd:Q3586605 ?p ?obj .


OPTIONAL{?p <http://schema.org/name> ?pName.}
OPTIONAL{?obj <http://schema.org/name> ?objName.}

} 

"""
 
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction'), ('obj', 'http://www.wikidata.org/entity/Q38')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q3586604')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q3586607')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q3586608')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q38')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q15146717')]
[('p', 'http://www.wikidata.org/prop/direct/P541'), ('pName', 'office contested'), ('obj', 'http://www.wikidata.org/entity/Q332711')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('pName', 'point in time'), ('ob

14

"Point in time" P585!! I can use it or I can use the property P580, "start time"

In [6]:
queryString = """
SELECT DISTINCT  ?person ?perName ?election ?time
WHERE { 
                            #president of italy
?person wdt:P39/wdt:P279* wd:Q332711  ;
        wdt:P3602 ?election . 
?election wdt:P991 ?person .
?election ?p ?time . 

?person <http://schema.org/name> ?perName.
FILTER (?p = wdt:P585 || ?p =wdt:P580)

} 
ORDER BY ASC (?time)
"""
 
print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1245'), ('perName', 'Enrico de Nicola'), ('election', 'http://www.wikidata.org/entity/Q3722093'), ('time', '1946-06-28T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q1245'), ('perName', 'Enrico de Nicola'), ('election', 'http://www.wikidata.org/entity/Q3722095'), ('time', '1947-06-26T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q1242'), ('perName', 'Luigi Einaudi'), ('election', 'http://www.wikidata.org/entity/Q3722096'), ('time', '1948-05-10T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q1241'), ('perName', 'Giovanni Gronchi'), ('election', 'http://www.wikidata.org/entity/Q3722097'), ('time', '1955-04-28T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q1241'), ('perName', 'Giovanni Gronchi'), ('election', 'http://www.wikidata.org/entity/Q3722097'), ('time', '1955-04-29T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q1240'), ('perName', 'Antonio Segni'), ('election', 'http://www.wi

16

Rewriting the query to find the first president

**Answer 3.1**

In [7]:
queryString = """

SELECT DISTINCT ?countryName ?presidentName ?president ?birth WHERE{


?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 

?elect wdt:P991 ?president .
?elect ?p ?t .

?president wdt:P569 ?birth .

?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?presidentName.

FILTER (?t = ?date)

            {
SELECT  (MIN(?time) as ?date) ?country
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P3602 ?election . 

?election wdt:P991 ?president .
?election wdt:P17 ?country .
?election ?p ?time . 

FILTER (?p = wdt:P585 || ?p =wdt:P580)

}
GROUP BY  ?date ?country
ORDER BY ASC (?country)


            }

        
}
ORDER BY ASC (?countryName)

"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Albania'), ('presidentName', 'Bujar Nishani'), ('president', 'http://www.wikidata.org/entity/Q57526'), ('birth', '1966-09-29T00:00:00Z')]
[('countryName', 'Argentina'), ('presidentName', 'Manuel Blanco Encalada'), ('president', 'http://www.wikidata.org/entity/Q1331'), ('birth', '1790-04-21T00:00:00Z')]
[('countryName', 'Argentina'), ('presidentName', 'Bernardino Rivadavia'), ('president', 'http://www.wikidata.org/entity/Q188891'), ('birth', '1780-05-20T00:00:00Z')]
[('countryName', 'Armenia'), ('presidentName', 'Levon Ter-Petrosyan'), ('president', 'http://www.wikidata.org/entity/Q612949'), ('birth', '1945-01-09T00:00:00Z')]
[('countryName', 'Austria'), ('presidentName', 'Adolf Schärf'), ('president', 'http://www.wikidata.org/entity/Q72898'), ('birth', '1890-04-20T00:00:00Z')]
[('countryName', 'Azerbaijan'), ('presidentName', 'Abulfaz Elchibey'), ('president', 'http://www.wikidata.org/entity/Q319912'), ('birth', '1938-06-24T00:00:00Z')]
[('countryName', 'Benin

82

In [22]:
queryString = """

SELECT (MIN(?time) as ?date) ?countryName
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P3602 ?election .   #partecipa all'election

?election wdt:P991 ?president .    #vince l'elezione
?election wdt:P17 ?country .       #del paese
?election ?p ?time . 

FILTER (?p = wdt:P585 || ?p =wdt:P580)
?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?prName.

}
GROUP BY  ?date ?countryName
ORDER BY ASC (?countryName)



"""

print("Results")
run_query(queryString)

Results
[('date', '2007-06-20T00:00:00Z'), ('countryName', 'Albania')]
[('date', '1826-01-01T00:00:00Z'), ('countryName', 'Argentina')]
[('date', '1991-10-17T00:00:00Z'), ('countryName', 'Armenia')]
[('date', '1963-04-28T00:00:00Z'), ('countryName', 'Austria')]
[('date', '1992-06-07T00:00:00Z'), ('countryName', 'Azerbaijan')]
[('date', '2006-03-05T00:00:00Z'), ('countryName', 'Benin')]
[('date', '2002-10-05T00:00:00Z'), ('countryName', 'Bosnia and Herzegovina')]
[('date', '2009-10-16T00:00:00Z'), ('countryName', 'Botswana')]
[('date', '1891-02-11T00:00:00Z'), ('countryName', 'Brazil')]
[('date', '1992-01-12T00:00:00Z'), ('countryName', 'Bulgaria')]
[('date', '1991-12-01T00:00:00Z'), ('countryName', 'Burkina Faso')]
[('date', '2010-06-28T00:00:00Z'), ('countryName', 'Burundi')]
[('date', '1964-11-03T00:00:00Z'), ('countryName', 'California')]
[('date', '1993-01-01T00:00:00Z'), ('countryName', 'Central African Republic')]
[('date', '1826-01-01T00:00:00Z'), ('countryName', 'Chile')]
[('da

90

------

Now I use one of the first query to find the number of president for each country. 

In [36]:
queryString = """

SELECT ?countryName (COUNT(DISTINCT (?president)) as ?num_presidents) 
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 

?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?presName.

}
GROUP BY  ?countryName ?num_presidents
ORDER BY ASC (?countryName)

"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Albania'), ('num_presidents', '10')]
[('countryName', 'Albanian Kingdom'), ('num_presidents', '1')]
[('countryName', 'Albanian Republic'), ('num_presidents', '1')]
[('countryName', 'Angola'), ('num_presidents', '3')]
[('countryName', 'Arab Kingdom of Syria'), ('num_presidents', '9')]
[('countryName', 'Argentina'), ('num_presidents', '60')]
[('countryName', 'Armenia'), ('num_presidents', '4')]
[('countryName', 'Austria'), ('num_presidents', '12')]
[('countryName', 'Austria-Hungary'), ('num_presidents', '10')]
[('countryName', 'Azerbaijan'), ('num_presidents', '5')]
[('countryName', "Ba'athist Iraq"), ('num_presidents', '1')]
[('countryName', 'Benin'), ('num_presidents', '5')]
[('countryName', 'Bolivia'), ('num_presidents', '3')]
[('countryName', 'Bosnia and Herzegovina'), ('num_presidents', '14')]
[('countryName', 'Botswana'), ('num_presidents', '5')]
[('countryName', 'Brazil'), ('num_presidents', '42')]
[('countryName', 'British America'), ('num_presidents', '

205

I'm not satisfied. Trying to change it a bit

In [37]:
queryString = """

SELECT ?countryName (COUNT(DISTINCT (?president)) as ?num_presidents) 
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P3602 ?election .   #partecipa all'election

?election wdt:P17 ?country . 

?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?presName.

}
GROUP BY  ?countryName ?num_presidents
ORDER BY ASC (?countryName)

"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Albania'), ('num_presidents', '2')]
[('countryName', 'Argentina'), ('num_presidents', '23')]
[('countryName', 'Armenia'), ('num_presidents', '4')]
[('countryName', 'Austria'), ('num_presidents', '7')]
[('countryName', 'Azerbaijan'), ('num_presidents', '4')]
[('countryName', 'Benin'), ('num_presidents', '2')]
[('countryName', 'Bosnia and Herzegovina'), ('num_presidents', '3')]
[('countryName', 'Botswana'), ('num_presidents', '1')]
[('countryName', 'Brazil'), ('num_presidents', '24')]
[('countryName', 'Bulgaria'), ('num_presidents', '5')]
[('countryName', 'Burkina Faso'), ('num_presidents', '2')]
[('countryName', 'Burundi'), ('num_presidents', '3')]
[('countryName', 'California'), ('num_presidents', '2')]
[('countryName', 'Central African Republic'), ('num_presidents', '3')]
[('countryName', 'Chile'), ('num_presidents', '31')]
[('countryName', 'Colombia'), ('num_presidents', '35')]
[('countryName', 'Comoros'), ('num_presidents', '3')]
[('countryName', 'Confedera

91

This last query seems to be more accurate for some countries, like Italy but for USA the number of president it is linked maybe to the property election. In the first query the USA presidents were higher because I was considering all the mandates. 

I stick to the first query

Finally I can answer to the second point of the third question

**Answer 3.2**

```
SELECT ?countryName (COUNT(DISTINCT (?president)) as ?num_presidents) 
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 

?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?presName.

}
GROUP BY  ?countryName ?num_presidents
ORDER BY ASC (?countryName)
```

--------

**4 - Is there a country that had at some point a woman as a president?**

Exploring P21 to obtain "FEMALE" entity

In [39]:
queryString = """

SELECT DISTINCT ?gender ?genName
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P21 ?gender  . 

?gender <http://schema.org/name> ?genName.

}


"""

print("Results")
run_query(queryString)

Results
[('gender', 'http://www.wikidata.org/entity/Q6581072'), ('genName', 'female')]
[('gender', 'http://www.wikidata.org/entity/Q6581097'), ('genName', 'male')]
[('gender', 'http://www.wikidata.org/entity/Q1052281'), ('genName', 'transgender female')]


3

FEMALE is Q6581072. Searching for countries with women in power

In [41]:
queryString = """

SELECT DISTINCT ?countryName ?presName
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  ;
            wdt:P21 wd:Q6581072 .


?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?presName.

}
ORDER BY ASC (?countryName)

"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Argentina'), ('presName', 'Cristina Fernández de Kirchner')]
[('countryName', 'Argentina'), ('presName', 'Isabel Martínez de Perón')]
[('countryName', 'Bosnia and Herzegovina'), ('presName', 'Biljana Plavšić')]
[('countryName', 'Bosnia and Herzegovina'), ('presName', 'Željka Cvijanović')]
[('countryName', 'Bosnia and Herzegovina'), ('presName', 'Srebrenka Golić')]
[('countryName', 'Brazil'), ('presName', 'Dilma Rousseff')]
[('countryName', 'Burundi'), ('presName', 'Sylvie Kinigi')]
[('countryName', 'Central African Republic'), ('presName', 'Catherine Samba-Panza')]
[('countryName', 'Costa Rica'), ('presName', 'Laura Chinchilla')]
[('countryName', 'Croatia'), ('presName', 'Ema Derossi-Bjelajac')]
[('countryName', 'Croatia'), ('presName', 'Kolinda Grabar-Kitarović')]
[('countryName', 'Duchy of Milan'), ('presName', 'Bona Sforza')]
[('countryName', 'Ecuador'), ('presName', 'Rosalía Arteaga')]
[('countryName', 'Estonia'), ('presName', 'Kersti Kaljulaid')]
[('count

51

Finally I can answer to the fourth question. Yes, there are countries with female president

**Answer 4**

```
SELECT DISTINCT ?countryName ?presName
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  ;
            wdt:P21 wd:Q6581072 .


?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?presName.

}
ORDER BY ASC (?countryName)

```
---------

**5.1 - What are the top-5 countries for number of presidents? Which countries had the least?**

Using the query used to answer to point 3 and filtering for the number of president. I print all the countries with only one president. Thery are 64 countries with 1 president

**Answer 5.1**

In [64]:
queryString = """
SELECT ?countryName (COUNT(DISTINCT (?president)) as ?num_presidents) 
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 

?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?presName.


}

GROUP BY  ?countryName ?num_presidents
HAVING ((COUNT(DISTINCT (?president))) < 2)

"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Kingdom of Serbs, Croats and Slovenes'), ('num_presidents', '1')]
[('countryName', 'Republic of the Congo (Léopoldville)'), ('num_presidents', '1')]
[('countryName', 'Syrian Federation'), ('num_presidents', '1')]
[('countryName', 'Kingdom of Romania'), ('num_presidents', '1')]
[('countryName', 'German Reich'), ('num_presidents', '1')]
[('countryName', 'Saudi Arabia'), ('num_presidents', '1')]
[('countryName', 'Ireland'), ('num_presidents', '1')]
[('countryName', 'Chinese Soviet Republic'), ('num_presidents', '1')]
[('countryName', "Ba'athist Iraq"), ('num_presidents', '1')]
[('countryName', 'Albanian Republic'), ('num_presidents', '1')]
[('countryName', 'United Provinces of Central America'), ('num_presidents', '1')]
[('countryName', 'Joseon'), ('num_presidents', '1')]
[('countryName', 'British America'), ('num_presidents', '1')]
[('countryName', 'Kingdom of Serbia'), ('num_presidents', '1')]
[('countryName', 'Western Sahara'), ('num_presidents', '1')]
[('coun

64

**answer 5.2**

Here we have the top 5

In [66]:
queryString = """
SELECT ?countryName (COUNT(DISTINCT (?president)) as ?num_presidents) 
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 

?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?presName.


}

GROUP BY  ?countryName ?num_presidents
ORDER BY DESC (?num_presidents)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Honduras'), ('num_presidents', '65')]
[('countryName', 'Argentina'), ('num_presidents', '60')]
[('countryName', 'Colombia'), ('num_presidents', '60')]
[('countryName', 'Mexico'), ('num_presidents', '59')]
[('countryName', 'United States of America'), ('num_presidents', '58')]


5

------------

**5.2 - For how many presidents we know a date of death in each country?**

I add to the previous graph pattern used to count the president the property P570, death date.

In [71]:
queryString = """
SELECT ?countryName (COUNT(DISTINCT (?death_date)) as ?num_dead_presidents) 
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?president wdt:P570 ?death_date . 

?country <http://schema.org/name> ?countryName.
?president <http://schema.org/name> ?presName.


}

GROUP BY  ?countryName ?num_dead_presidents
ORDER BY DESC (?num_dead_presidents)
#LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Argentina'), ('num_dead_presidents', '61')]
[('countryName', 'Colombia'), ('num_dead_presidents', '57')]
[('countryName', 'Venezuela'), ('num_dead_presidents', '56')]
[('countryName', 'Honduras'), ('num_dead_presidents', '53')]
[('countryName', 'Mexico'), ('num_dead_presidents', '52')]
[('countryName', 'Paraguay'), ('num_dead_presidents', '50')]
[('countryName', 'Haiti'), ('num_dead_presidents', '50')]
[('countryName', 'Guatemala'), ('num_dead_presidents', '44')]
[('countryName', 'Peru'), ('num_dead_presidents', '43')]
[('countryName', 'United States of America'), ('num_dead_presidents', '43')]
[('countryName', 'Brazil'), ('num_dead_presidents', '38')]
[('countryName', 'Costa Rica'), ('num_dead_presidents', '35')]
[('countryName', 'Chile'), ('num_dead_presidents', '33')]
[('countryName', 'El Salvador'), ('num_dead_presidents', '33')]
[('countryName', 'Spain'), ('num_dead_presidents', '27')]
[('countryName', 'Cuba'), ('num_dead_presidents', '26')]
[('countryNam

184

Italy correctly display 10 died presidents. 

---------

**5.3 - Which are the professions of different presidents?**

Find the profession P106

In [74]:
queryString = """
SELECT DISTINCT ?occupation ?occName
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?president wdt:P106 ?occupation . 

?country <http://schema.org/name> ?countryName.
?occupation <http://schema.org/name> ?occName.


}
ORDER BY ASC (?occName)

"""

print("Results")
run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q250867'), ('occName', 'Catholic priest')]
[('occupation', 'http://www.wikidata.org/entity/Q1377734'), ('occName', 'Chernobyl liquidator')]
[('occupation', 'http://www.wikidata.org/entity/Q868715'), ('occName', 'Conducător')]
[('occupation', 'http://www.wikidata.org/entity/Q860918'), ('occName', 'Esperantist')]
[('occupation', 'http://www.wikidata.org/entity/Q37516163'), ('occName', 'Militar')]
[('occupation', 'http://www.wikidata.org/entity/Q5784164'), ('occName', 'Palace janitor')]
[('occupation', 'http://www.wikidata.org/entity/Q466956'), ('occName', 'President of Chile')]
[('occupation', 'http://www.wikidata.org/entity/Q5400907'), ('occName', 'Presidente')]
[('occupation', 'http://www.wikidata.org/entity/Q3400985'), ('occName', 'academic')]
[('occupation', 'http://www.wikidata.org/entity/Q15253558'), ('occName', 'activist')]
[('occupation', 'http://www.wikidata.org/entity/Q33999'), ('occName', 'actor')]
[('occupation', 'http:/

208

**Answer 5.3**
Filtering for the ones different from political domanin

In [15]:
queryString = """
SELECT DISTINCT ?occupation ?occName
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?president wdt:P106 ?occupation . 

?country <http://schema.org/name> ?countryName.
?occupation <http://schema.org/name> ?occName.

FILTER ((!REGEX(?occName, ".*politi.*")) AND (!REGEX(?occName, ".*minis.*")))
FILTER ((!REGEX(?occName, ".*[Pp]resid.*")))
}
ORDER BY ASC (?occName)

"""

print("Results")
run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q250867'), ('occName', 'Catholic priest')]
[('occupation', 'http://www.wikidata.org/entity/Q1377734'), ('occName', 'Chernobyl liquidator')]
[('occupation', 'http://www.wikidata.org/entity/Q868715'), ('occName', 'Conducător')]
[('occupation', 'http://www.wikidata.org/entity/Q860918'), ('occName', 'Esperantist')]
[('occupation', 'http://www.wikidata.org/entity/Q37516163'), ('occName', 'Militar')]
[('occupation', 'http://www.wikidata.org/entity/Q5784164'), ('occName', 'Palace janitor')]
[('occupation', 'http://www.wikidata.org/entity/Q3400985'), ('occName', 'academic')]
[('occupation', 'http://www.wikidata.org/entity/Q15253558'), ('occName', 'activist')]
[('occupation', 'http://www.wikidata.org/entity/Q33999'), ('occName', 'actor')]
[('occupation', 'http://www.wikidata.org/entity/Q47227062'), ('occName', 'agitator')]
[('occupation', 'http://www.wikidata.org/entity/Q26252897'), ('occName', 'agricultural economist')]
[('occupation', 'h

191

Counting the work of each politician with the previous discovered properties


In [16]:
queryString = """
SELECT (COUNT(DISTINCT (?occName))AS ?num_works) ?presName
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?president wdt:P106 ?occupation . 

?country <http://schema.org/name> ?countryName.
?occupation <http://schema.org/name> ?occName.
?president <http://schema.org/name> ?presName.

FILTER ((!REGEX(?occName, ".*politi.*")) AND (!REGEX(?occName, ".*minis.*")))
FILTER ((!REGEX(?occName, ".*[Pp]resid.*")))

}
GROUP BY ?work ?presName
ORDER BY DESC (?num_works)

"""

print("Results")
run_query(queryString)

Results
[('num_works', '8'), ('presName', 'Mao Zedong')]
[('num_works', '7'), ('presName', 'Lennart Meri')]
[('num_works', '7'), ('presName', 'Ion Iliescu')]
[('num_works', '6'), ('presName', 'Kazys Grinius')]
[('num_works', '6'), ('presName', 'Josip Vidmar')]
[('num_works', '6'), ('presName', 'Luis Cordero Crespo')]
[('num_works', '6'), ('presName', 'Miguel Antonio Caro Tobar')]
[('num_works', '5'), ('presName', 'Bartolomé Mitre')]
[('num_works', '5'), ('presName', 'Sebastián Piñera')]
[('num_works', '5'), ('presName', 'Domingo Faustino Sarmiento')]
[('num_works', '5'), ('presName', 'Danilo Türk')]
[('num_works', '5'), ('presName', 'Francisco García Calderón')]
[('num_works', '5'), ('presName', 'Luigi Einaudi')]
[('num_works', '5'), ('presName', 'Franz Jonas')]
[('num_works', '5'), ('presName', 'Julius Nyerere')]
[('num_works', '5'), ('presName', 'Armen Sarkissian')]
[('num_works', '5'), ('presName', 'Rafael Núñez')]
[('num_works', '5'), ('presName', 'Konstantinos Tsatsos')]
[('num_wo

1095

**Answer 5.3**

There are about 1000 presidents with non-political jobs.

In [17]:
queryString = """
SELECT (COUNT(DISTINCT (?presName)) as ?num_pres) 
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  .
?president wdt:P27 ?country  . 
?president wdt:P106 ?occupation . 

?country <http://schema.org/name> ?countryName.
?occupation <http://schema.org/name> ?occName.
?president <http://schema.org/name> ?presName.

FILTER ((!REGEX(?occName, ".*politi.*")) AND (!REGEX(?occName, ".*minis.*")))
FILTER ((!REGEX(?occName, ".*[Pp]resid.*")))

}

"""

print("Results")
run_query(queryString)

Results
[('num_pres', '1095')]


1